## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [1]:
# Your code here - remember to use markdown cells for comments as well!

In [2]:
import json
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from collections import defaultdict
import seaborn as sns
import csv
import sqlite3
import pandasql

In [3]:
bom_gross_df = pd.read_csv('data/bom.movie_gross.csv')
bom_gross_df.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [4]:
tmdb_df = pd.read_csv('data/tmdb.movies.csv')
tmdb_df.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [5]:
tn_budgets_df = pd.read_csv('data/tn.movie_budgets.csv')
tn_budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [6]:
# Map of genres for identification in tmdb file
genre_map = {28:'Action', 16:'Animation', 99:'Documentary', 18:'Drama', 10751:'Family', 14:'Fantasy', 36:'History', 35:'Comedy', 
             10752:'War', 80:'Crime', 10402:'Musical', 9648:'Mystery', 10749:'Romance', 878:'Science Fiction', 27:'Horror', 
             10770:'TV movie', 53:'Thriller', 37:'Western', 12:'Adventure'}

In [64]:
#remove '[', ']', and ',' from genre_ids
tmdb_df['genre_ids'] = tmdb_df['genre_ids'].str.replace('[\[\]\,]', '', regex=True)

In [66]:
#map genre_id to genre

def map_genres(ids_string, genre_map):
    return ' '.join([genre_map[int(genre_id)] for genre_id in ids_string.split() if int(genre_id) in genre_map])

tmdb_df['genre_names'] = tmdb_df['genre_ids'].apply(lambda x: map_genres(x, genre_map))
tmdb_df = tmdb_df.drop(['genre_ids'], axis=1)


KeyError: 'genre_ids'

In [ ]:
tmdb_df.head()

In [21]:
tn_budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [45]:
# Converting worldwide_gross to float

remove_dollar = lambda x: float(x.replace('$', ''))

tn_budgets_df['worldwide_gross'] = tn_budgets_df['worldwide_gross'].str.replace(',', '')
tn_budgets_df['worldwide_gross'] = tn_budgets_df['worldwide_gross'].apply(remove_dollar)
tn_budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625",2.776345e+09
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875",1.045664e+09
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350",1.497624e+08
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868",1.403014e+09
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382",1.316722e+09


In [51]:
# Converting worldwide_gross to float

remove_dollar = lambda x: float(x.replace('$', ''))

tn_budgets_df['production_budget'] = tn_budgets_df['production_budget'].str.replace(',', '')
tn_budgets_df['production_budget'] = tn_budgets_df['production_budget'].apply(remove_dollar)
tn_budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000.0,"$760,507,625",2.776345e+09
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,"$241,063,875",1.045664e+09
2,3,"Jun 7, 2019",Dark Phoenix,350000000.0,"$42,762,350",1.497624e+08
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000.0,"$459,005,868",1.403014e+09
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,"$620,181,382",1.316722e+09


In [52]:
tn_budgets_df.sort_values(by= ['worldwide_gross'], ascending = False)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000.0,"$760,507,625",2.776345e+09
42,43,"Dec 19, 1997",Titanic,200000000.0,"$659,363,944",2.208208e+09
5,6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,306000000.0,"$936,662,225",2.053311e+09
6,7,"Apr 27, 2018",Avengers: Infinity War,300000000.0,"$678,815,482",2.048134e+09
33,34,"Jun 12, 2015",Jurassic World,215000000.0,"$652,270,625",1.648855e+09
...,...,...,...,...,...,...
5474,75,"Dec 31, 2005",Insomnia Manica,500000.0,$0,0.000000e+00
5473,74,"Jul 17, 2012",Girls Gone Dead,500000.0,$0,0.000000e+00
5472,73,"Apr 3, 2012",Enter Nowhere,500000.0,$0,0.000000e+00
5471,72,"Dec 31, 2010",Drones,500000.0,$0,0.000000e+00


In [59]:
tn_budgets_df['ww_profit_ratio'] = tn_budgets_df['worldwide_gross'] / tn_budgets_df['production_budget']

In [61]:
tn_budgets_df['ww_profit'] = tn_budgets_df['worldwide_gross'] - tn_budgets_df['production_budget']

In [62]:
tn_budgets_df.sort_values(by= ['ww_profit_ratio'], ascending = False).head(20)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,ww_profit,ww_profit_ratio
5745,46,"Jun 30, 1972",Deep Throat,25000.0,"$45,000,000",45000000.0,44975000.0,1800.000000
5613,14,"Mar 21, 1980",Mad Max,200000.0,"$8,750,000",99750000.0,99550000.0,498.750000
5492,93,"Sep 25, 2009",Paranormal Activity,450000.0,"$107,918,810",194183034.0,193733034.0,431.517853
5679,80,"Jul 10, 2015",The Gallows,100000.0,"$22,764,410",41656474.0,41556474.0,416.564740
5406,7,"Jul 14, 1999",The Blair Witch Project,600000.0,"$140,539,099",248300000.0,247700000.0,413.833333
5709,10,"May 7, 2004",Super Size Me,65000.0,"$11,529,368",22233808.0,22168808.0,342.058585
5346,47,"Aug 13, 1942",Bambi,858000.0,"$102,797,000",268000000.0,267142000.0,312.354312
5773,74,"Feb 26, 1993",El Mariachi,7000.0,"$2,040,920",2041928.0,2034928.0,291.704000
5676,77,"Oct 1, 1968",Night of the Living Dead,114000.0,"$12,087,064",30087064.0,29973064.0,263.921614
5210,11,"Nov 21, 1976",Rocky,1000000.0,"$117,235,147",225000000.0,224000000.0,225.000000
